In [18]:
import numpy as np
import pandas as pd
from utils.formats import load_hdf

import torch
from torch import nn
import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

from datasets import Dataset
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, MobileBertForSequenceClassification 
from transformers import TrainingArguments, Trainer
from transformers.configuration_utils import PretrainedConfig

### Huggingface model - [MobileBERT](https://huggingface.co/docs/transformers/model_doc/mobilebert#transformers.MobileBertForSequenceClassification)

* Input Embedding Dimensionality cannot be too big. 
* Standard Flavours of BERT-based transformer models have input dim of 768. PPMi + Retrofitting takes too long to produce input embedding vectors.

In [19]:
MODEL_NAME = "lordtt13/emo-mobilebert"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = MobileBertForSequenceClassification.from_pretrained(MODEL_NAME) # Specify num_labels for your task
# model = AlbertModel.from_pretrained(MODEL_NAME, torch_dtype=torch.float16) # Specify num_labels for your task

vocab = tokenizer.get_vocab()
vocab_size = len(vocab)
embedding_dim = model.config.embedding_size  #hidden_size
print(f"Tokenizer Vocab Size: {vocab_size}\nEmbedding Dimensionality: {embedding_dim}")
print(f"Vocab:\n{vocab}")

Tokenizer Vocab Size: 2016
Embedding Dimensionality: 128
Vocab:
{'##6': 75, 'buddy': 1792, '##ow': 101, 'bo': 557, '##ass': 624, 'des': 1039, 'rain': 1911, 'yep': 1340, '##self': 535, 'eas': 1644, 'best': 479, 'romantic': 1691, 'once': 1489, '##ingfacewithsun': 1648, 'you': 80, 'cheat': 1015, 'says': 1683, 'wrong': 583, 'last': 867, 'type': 1093, 'problems': 1771, 'dist': 1104, 'where': 302, 'rom': 1410, 'dare': 1813, '##ps': 1066, 'might': 1417, 'problem': 684, '##ge': 277, '##ware': 1433, 'absol': 1825, 'li': 127, 'smilingcatfacewithhearteyes': 1684, 'fem': 1705, 'car': 1181, 'mil': 1797, 'sc': 653, 'gi': 1046, '##severingface': 1712, 'didn': 526, '##ally': 417, '##ck': 199, 'grinn': 363, 'proba': 1436, 'dance': 1267, '##winkingface': 1678, 'ref': 1985, 'sub': 1699, 'pain': 948, 'true': 881, '##grinningfacewithsweat': 1304, 'laugh': 682, 'new': 593, '##ant': 568, '##at': 82, 'girls': 1227, 'thn': 1354, 'lie': 1299, 'a': 16, 'got': 478, 'makes': 1025, '##and': 303, '##rown': 872, 'fav

#### Load Retrofitted PPMI word embeddings for MobileBERT (dim=128)

* Since index of input word embedding matrix after retrofitting can contain multiple words due to edge connections, data cleaning is required to process the index such that one word remains (e.g. `/c/en/president/n/wn/person` --> `president`)
* This step required to match ALBERT tokenizer's vocab so that the corresponding input word embedding can be identified and modified.

In [20]:
input_embedding = load_hdf("data/conceptnet_api/retrofit/retrofitted-albert-128")
input_embedding_df = input_embedding.reset_index()
input_embedding_df['vocab'] = input_embedding_df['index'].str.extract(r'/c/en/(\w+)/?')
input_embedding_df.head()

,index,0,1,2,3,4,5,6,7,8,...,119,120,121,122,123,124,125,126,127,vocab
0,/c/en/help_child,-0.000251,-0.010002,-0.006968,0.000776,0.004100,0.001745,-0.005115,-0.001407,0.000003,...,-0.043209,-0.044639,0.018009,-0.033039,-0.048886,0.055936,0.052766,0.071421,0.092698,help_child
1,/c/en/adult,-0.000234,-0.007938,-0.006174,0.000723,0.003105,0.001632,-0.003933,-0.001316,0.000002,...,-0.045798,-0.042103,0.058996,-0.069027,-0.040135,0.050881,0.049507,0.067248,0.077322,adult
2,/c/en/man,-0.000250,-0.005036,0.014826,0.000771,-0.018620,0.001739,0.024272,-0.001402,0.000003,...,-0.081502,-0.044615,0.222954,-0.129706,-0.030820,0.051938,0.053575,0.056078,0.090643,man
3,/c/en/sign_contract,-0.000251,-0.010131,-0.006857,0.000780,0.004132,0.001748,-0.005139,-0.001410,0.000003,...,-0.043298,-0.044735,0.018026,-0.033055,-0.048998,0.056059,0.052880,0.071576,0.092903,sign_contract
4,/c/en/dress_herself,-0.000251,-0.010131,-0.006857,0.000780,0.004132,0.001748,-0.005139,-0.001410,0.000003,...,-0.043298,-0.044735,0.018026,-0.033055,-0.048998,0.056059,0.052880,0.071576,0.092903,dress_herself


In [21]:
# convert retrofit ppimi word embedding into numpy matrix form
input_embedding_matrix = input_embedding.to_numpy()
print(input_embedding_matrix.shape)
input_embedding_matrix

(4081, 128)


array([[-0.00025054, -0.01000199, -0.006968  , ...,  0.05276616,
         0.07142138,  0.09269828],
       [-0.00023437, -0.00793784, -0.00617399, ...,  0.04950697,
         0.06724799,  0.07732166],
       [-0.00024969, -0.0050365 ,  0.01482603, ...,  0.05357489,
         0.0560779 ,  0.0906431 ],
       ...,
       [-0.00021273, -0.00120958, -0.00064862, ...,  0.04435906,
         0.05975946,  0.09372985],
       [-0.00021273, -0.00120958, -0.00064862, ...,  0.04435906,
         0.05975946,  0.09372985],
       [-0.00021273, -0.00120958, -0.00064862, ...,  0.04435906,
         0.05975946,  0.09372985]], shape=(4081, 128), dtype=float32)

In [22]:
# Get Embedding Weights of ALBERT model
# embedding_layer = model.embeddings.word_embeddings # For AlbertModel object
mobilebert_model = model._modules['mobilebert']
embedding_layer = mobilebert_model.embeddings.word_embeddings

# torch.no_grad() to avoid tracking gradients
with torch.no_grad():
    embedding_matrix = embedding_layer.weight.clone() # Clone to avoid modifying original

default_embedding_matrix = embedding_matrix.cpu().numpy()
print(default_embedding_matrix.shape)
default_embedding_matrix

(2016, 128)


array([[-0.01316209,  0.00857995,  0.01150196, ..., -0.01960188,
         0.01102599,  0.0168985 ],
       [-0.02624886,  0.00065748,  0.00497713, ...,  0.00576143,
        -0.01545058,  0.00221606],
       [ 0.03140381, -0.02327314, -0.0265863 , ...,  0.00643379,
         0.03365475, -0.03251279],
       ...,
       [ 0.01972965,  0.01448449, -0.02070585, ...,  0.0077508 ,
        -0.02492797,  0.00269021],
       [-0.01042436,  0.01346194, -0.04289725, ...,  0.01183029,
         0.00020893, -0.01214224],
       [ 0.01525993, -0.04257187, -0.01507291, ...,  0.00786374,
        -0.01781924,  0.00746477]], shape=(2016, 128), dtype=float32)

#### Logic to modify default word embedding

* 

In [23]:
modified_words = input_embedding_df['vocab'].to_list()

def _tokenize(word:str):
    # Handle case sensitivity based on the tokenizer
    processed_word = word.lower() if tokenizer.do_lower_case else word

    # Tokenize the word - it might split into subwords
    tokens = tokenizer.tokenize(processed_word)
    return tokens

modification_cache = dict() # store idx and words that were modified. 
for idx, word in enumerate(modified_words):

    tokens = _tokenize(word)

    if len(tokens) == 1:

        token = tokens[0]

        embedding_idx = vocab[token]

        modification_cache['/c/en/' + word] = embedding_idx

        new_embedding_array = input_embedding_matrix[idx]

        default_embedding_matrix[embedding_idx] = new_embedding_array

# Convert to PyTorch/TensorFlow tensor
new_embedding_tensor = torch.tensor(default_embedding_matrix, dtype=torch.float16)

print(new_embedding_tensor.shape)
print(new_embedding_tensor)

assert embedding_layer.weight.shape == new_embedding_tensor.shape, \
    f"Shape mismatch: Model expects {embedding_layer.weight.shape}, got {new_embedding_tensor.shape}"

torch.Size([2016, 128])
tensor([[-0.0132,  0.0086,  0.0115,  ..., -0.0196,  0.0110,  0.0169],
        [-0.0002, -0.0012, -0.0005,  ...,  0.0473,  0.0490,  0.0898],
        [ 0.0314, -0.0233, -0.0266,  ...,  0.0064,  0.0337, -0.0325],
        ...,
        [ 0.0197,  0.0145, -0.0207,  ...,  0.0078, -0.0249,  0.0027],
        [-0.0104,  0.0135, -0.0429,  ...,  0.0118,  0.0002, -0.0121],
        [ 0.0153, -0.0426, -0.0151,  ...,  0.0079, -0.0178,  0.0075]],
       dtype=torch.float16)


In [24]:
# Replace the weights (ensure device placement is correct if using GPU)
with torch.no_grad(): # Prevent tracking this operation in gradient history
     embedding_layer.weight.copy_(new_embedding_tensor) # In-place copy is safer

# Make sure the embedding layer is trainable (usually true by default after loading)
embedding_layer.weight.requires_grad = True

### Model Finetuning - Sequence Classification task

* Dataset: [`flax-sentence-embeddings/Gender_Bias_Evaluation_Set`](https://huggingface.co/datasets/flax-sentence-embeddings/Gender_Bias_Evaluation_Set)
* Relatively small dataset for Sequence Classification task (1584 rows)

In [25]:
df = pd.read_csv("hf://datasets/flax-sentence-embeddings/Gender_Bias_Evaluation_Set/bias_evaluation.csv")
df['labels'] = df['stereotypical_gender'].apply(lambda x: 1 if x == "male" else 0)
df.head()

,Unnamed: 0,base_sentence,occupation,stereotypical_gender,male_sentence,female_sentence,labels
0,0,The developer argued with the designer because...,developer,male,He argued with the designer because he did not...,She argued with the designer because she did n...,1
1,1,The developer argued with the designer because...,designer,female,The developer argued with him because his idea...,The developer argued with her because her idea...,0
2,2,The mechanic gave the clerk a present because ...,mechanic,male,He gave the clerk a present because he won the...,She gave the clerk a present because she won t...,1
3,3,The mechanic gave the clerk a present because ...,clerk,female,The mechanic gave him a present because it was...,The mechanic gave her a present because it was...,0
4,4,The mover said thank you to the housekeeper be...,mover,male,He said thank you to the housekeeper because h...,She said thank you to the housekeeper because ...,1


In [26]:
# Convert dataset into Huggingface Dataset object with train-test split of 80:20
datasets = Dataset.from_pandas(df).train_test_split(test_size=0.2)

train_dataset = datasets["train"]
val_dataset = datasets["test"]

train_dataset[0]

{'Unnamed: 0': 581,
 'base_sentence': 'The analyst sent a report to the editor because he put in a request yesterday.',
 'occupation': 'editor',
 'stereotypical_gender': 'female',
 'male_sentence': 'The analyst sent a report to him because he put in a request yesterday.',
 'female_sentence': 'The analyst sent a report to her because she put in a request yesterday.',
 'labels': 0}

In [27]:
# Tokenize the `base_sentence` column so that it can be used as input to finetune ALBERT
def tokenize_function(examples):
    return tokenizer(examples["base_sentence"], padding="max_length", truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 317/317 [00:00<00:00, 3990.19 examples/s]


In [28]:
# Format the dataset for PyTorch - Remove columns not needed by the model
cols_to_remove = ["Unnamed: 0", "base_sentence", "occupation", "male_sentence", "female_sentence", "stereotypical_gender"]
tokenized_train_dataset = tokenized_train_dataset.remove_columns(cols_to_remove)
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(cols_to_remove)

# # Rename the 'stereotypical_gender' column to 'labels' (expected by Trainer)
# tokenized_train_dataset = tokenized_train_dataset.rename_column("stereotypical_gender", "labels")
# tokenized_eval_dataset = tokenized_eval_dataset.rename_column("stereotypical_gender", "labels")

# Set format to PyTorch tensors
tokenized_train_dataset.set_format("torch")
tokenized_eval_dataset.set_format("torch")


In [29]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Logits are the raw output scores from the model, shape (batch_size, num_labels)
    # Labels are the ground truth, shape (batch_size,)
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [30]:
training_args = TrainingArguments(
    output_dir="./results",             # Directory to save model checkpoints and logs
    num_train_epochs=1,                 # Reduced for quick demonstration; use more epochs (e.g., 3-5) for real tasks
    per_device_train_batch_size=8,      # Adjust based on your GPU memory
    per_device_eval_batch_size=8,       # Adjust based on your GPU memory
    warmup_steps=100,                   # Number of steps for linear warmup
    weight_decay=0.01,                  # Regularization strength
    logging_dir="./logs",               # Directory for TensorBoard logs
    logging_steps=50,                   # Log metrics every 50 steps
    # evaluation_strategy="epoch",        # Evaluate performance at the end of each epoch
    # save_strategy="epoch",              # Save model checkpoint at the end of each epoch
    # load_best_model_at_end=True,        # Load the best model found during training at the end
    metric_for_best_model="accuracy",   # Metric used to determine the best model
    greater_is_better=True,             # Accuracy should be maximized
    report_to="tensorboard",            # Report logs to TensorBoard (can add "wandb" etc.)
    # push_to_hub=False,                # Set to True to push model to Hugging Face Hub
    fp16=torch.cuda.is_available(),     # Use mixed precision training if CUDA is available
)

trainer = Trainer(
    model=model,                        # The model to train (potentially with custom embeddings)
    args=training_args,                 # Training arguments defined above
    train_dataset=tokenized_train_dataset, # Training dataset
    eval_dataset=tokenized_eval_dataset,   # Evaluation dataset
    tokenizer=tokenizer,                # Tokenizer used for data collation (handles padding dynamically if needed)
    compute_metrics=compute_metrics,    # Function to compute evaluation metrics
    # Optional: Data collator can optimize padding
    # data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

/tmp/ipykernel_74454/2816268275.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [31]:
train_result = trainer.train()

trainer.save_model()  # Saves the tokenizer too
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

# 6. Evaluate the Final Model
print("Evaluating the final model...")
eval_metrics = trainer.evaluate()
print(f"Evaluation Metrics: {eval_metrics}")
trainer.log_metrics("eval", eval_metrics)
trainer.save_metrics("eval", eval_metrics)

Step,Training Loss
50,1.676700
100,0.704800
150,0.536800


/home/bernard/miniconda3/envs/cs4248_project/lib/python3.11/site-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 128}
  warnings.warn(
/home/bernard/miniconda3/envs/cs4248_project/lib/python3.11/site-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will b

***** train metrics *****
  epoch                    =        1.0
  total_flos               =    74950GF
  train_loss               =     0.9402
  train_runtime            = 0:00:40.88
  train_samples_per_second =     30.988
  train_steps_per_second   =      3.889
Evaluating the final model...


Evaluation Metrics: {'eval_loss': 0.42843079566955566, 'eval_accuracy': 0.8107255520504731, 'eval_runtime': 2.5329, 'eval_samples_per_second': 125.155, 'eval_steps_per_second': 15.792, 'epoch': 1.0}
***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.8107
  eval_loss               =     0.4284
  eval_runtime            = 0:00:02.53
  eval_samples_per_second =    125.155
  eval_steps_per_second   =     15.792


In [32]:
# Access the embedding layer again (use the same path as in Step 4)
final_embedding_layer = mobilebert_model.embeddings.word_embeddings

# Get the weights
final_embeddings_tensor = final_embedding_layer.weight.data

# Convert to NumPy if desired (and move to CPU if on GPU)
final_embeddings_numpy = final_embeddings_tensor.cpu().numpy()
print(final_embeddings_numpy.shape)
final_embeddings_numpy

(2016, 128)


array([[-0.01316017,  0.00858272,  0.01150468, ..., -0.01960675,
         0.01102403,  0.01689079],
       [-0.00021168, -0.00105971, -0.00010905, ...,  0.04698017,
         0.04900245,  0.08960536],
       [ 0.03084011, -0.02333198, -0.02677003, ...,  0.00656562,
         0.0337524 , -0.03196188],
       ...,
       [ 0.01972882,  0.01448763, -0.02070535, ...,  0.00775115,
        -0.02493186,  0.00268925],
       [-0.01042134,  0.01345772, -0.04290606, ...,  0.01183271,
         0.00020897, -0.01214551],
       [ 0.01525817, -0.04257037, -0.01507506, ...,  0.00786556,
        -0.01782154,  0.00746505]], shape=(2016, 128), dtype=float32)

In [33]:
conceptnet_finetune_embeddings = dict()

for concept, idx in modification_cache.items():
    conceptnet_finetune_embeddings[concept] = final_embeddings_numpy[idx].tolist()

conceptnet_finetune_embeddings_df = pd.DataFrame.from_dict(conceptnet_finetune_embeddings, orient='index')
print(conceptnet_finetune_embeddings_df.shape)
conceptnet_finetune_embeddings_df.head()

(134, 128)


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
/c/en/man,-0.000658,0.001006,-0.005877,0.001038,0.000685,0.001771,-0.001230,-0.001466,-0.000417,0.001877,...,0.007122,-0.039805,-0.039871,-0.040023,-0.541082,0.019830,0.027921,0.050228,0.058784,0.076488
/c/en/work,-0.000922,-0.007203,-0.002126,0.003844,0.004400,0.008029,-0.001586,-0.006528,0.000183,0.011738,...,0.152243,-0.146789,-0.198548,-0.264450,0.271315,-0.278589,0.268105,0.236308,0.317686,0.498062
/c/en/woman,-0.000216,-0.001651,0.000637,0.000704,0.000750,0.001503,-0.000081,-0.001212,0.000002,0.002205,...,0.051054,-0.015495,-0.036223,-0.207023,-0.737764,0.012840,0.036711,0.045927,0.059537,0.090695
/c/en/person,-0.000246,-0.009055,0.008888,0.010461,0.160612,0.002951,-0.126760,-0.002315,0.000220,0.004458,...,0.065617,-0.039346,-0.074550,0.252103,0.046779,-0.087000,0.096606,0.084621,0.084979,0.167017
/c/en/child,-0.000291,-0.009948,-0.007065,0.000934,-0.001176,0.002027,0.001944,-0.001635,0.000003,0.002977,...,0.067685,-0.138911,-0.056181,0.537576,-0.062284,-0.044218,0.065610,0.059629,0.079831,0.085812


In [34]:
conceptnet_finetune_embeddings_df.to_hdf(path_or_buf="data/ml_finetune/retrofitted-custom-mobilebert-128", key='mat', encoding='utf-8')